Install and import required module

In [42]:
# install modules
!pip install requests
!pip install beautifulsoup4

import requests
from bs4 import BeautifulSoup
import re

Configuration

In [5]:
#configuration

SITE_URL = 'https://www.imdb.com'
# to act like user / not robot
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}


Gathering data from page and export to CSV file

In [ ]:
URL = SITE_URL + "/search/title/?groups=top_250&sort=user_rating,desc"
movie_list = []
while True :
  page = requests.get(URL, headers=headers)
  soup = BeautifulSoup(page.content, 'html.parser')
  content_headers = soup.find_all("div", {"class": "lister-item-content"})
  for content in content_headers :
     
    move_item = []
    #movie name
    move_item.append(content.find("h3", {"class": "lister-item-header"}).find("a").text)
    #movie time
    move_item.append(content.find("span", {"class": "runtime"}).text)
    #moview user rating
    move_item.append(content.find("div", {"class": "ratings-imdb-rating"})['data-value'])
    #moview metasore
    move_item.append(content.find("span", {"class": "metascore"}).text.strip() if content.find("span", {"class": "metascore"}) else None)
    #moview total vote
    move_item.append(re.search(r'\d{1,3}(,\d{3})*', content.find("p", {"class": "sort-num_votes-visible"}).text).group())
    movie_list.append(move_item)
  # check next page available or not
  next_page = soup.find("a", {"class": "next-page"}) 
  if next_page != None : 
    URL = SITE_URL + next_page['href']
  else :
    break
df = pd.DataFrame(movie_list, columns =['Name', 'Time length','IMDb Rating','Meta Score','Total Votes'])
df.to_csv('top_movie_data.csv',index=False)